In [ ]:
# PACKAGE INSTALLATION

%pip install jupyter stanza dotenv psycopg2

In [ ]:
# LOCAL IMPORTS

import importlib
import romanizer
import db

importlib.reload(romanizer)     # Reload romanizer from korean
importlib.reload(db)

In [ ]:
# DATABASE

import os
from dotenv import load_dotenv
from supabase import create_client, Client
from psycopg2.extensions import connection

load_dotenv(override=True)

# url: str = os.environ.get("SUPABASE_URL")
# key: str = os.environ.get("SUPABASE_KEY")

# conn: Client = create_client(url, key)

conn: connection = db.connect()

In [ ]:
# TABLE CREATION

db.create_tables(conn)

In [ ]:
# MODEL DOWNLOAD

import stanza

language = 'ko'
stanza.download(language)       # Download model
nlp = stanza.Pipeline(language) # Initialize neural pipeline

In [ ]:
# PHRASE TOKENIZATION
phrase = "무엇을 먹을 거예요?"

doc = nlp(phrase)
dict = doc.to_dict()

print(doc.sentences)

In [ ]:
from color import blue, bold, cyan, green, purple, red


for sentences in dict:
    print("Phrase: ", phrase)
    for words in sentences:
        print(bold(cyan("Text: ")), words['text'], '|', blue(romanizer.hangul(words['text'])))

        upos = db.get_upos(conn, words['upos'])
        print(green("Part of Speech: "), upos)

        print(red("Morphemes: "))
        morphemes = words['lemma'].split('+')

        tags = words['xpos'].split('+')
        xpos = db.get_xpos(conn, tags)

        morphemes_info = db.get_morphemes(conn, morphemes, tags)

        for morpheme, tag, xpos_label, info in zip(morphemes, tags, xpos, morphemes_info):
            print(f"    '{morpheme}': '{tag} => {xpos_label.title()}' | {bold(red('Translation'))}: {info['translation']}")

        translation = db.get_translation(conn, words['text'], words['upos'], morphemes_info)
        print(purple("Translation: "), translation)
        print("─"*30, '\n')
